In [2]:
# For using custom module.
import sys
sys.path.append('../custom_library')

In [3]:
import pymysql.cursors
import pandas as pd
import numpy as np
# import connect_to_db as cn
import json
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import csv
import parmap

In [4]:
def check_body(document):
    p = re.compile('[a-zA-Z]')
    for text in document[:]:
        # 특수문자, 이모지 등으로만 이루어진 text를 거르기 위한 code.
        if p.search(text) is None:
            document.remove(text)
        # 특수문자와 알파벳 하나(ex. :p)로만 이루어진 text를 거르기 위한 code.
        elif len(re.sub('[^a-zA-Z]',' ',text).strip()) == 1:
            document.remove(text)
        else:
            text_list = text.split('.')
            length = len(text_list)
            for i in range(length):
                if len(text_list[i]) > 1:
                    break
                if length - 1 == i:
                    document.remove(text)
    
    return document

In [6]:
result_df = pd.read_csv('seeker_satisfaction_1000_thread.csv')
# print(result_df)
post_ids = list(result_df['post_id'])
post_contents = list(result_df['post_content'])
post_titles = list(result_df['post_title'])

post_ids_contents = []
post_ids_titles = []

for i in range(len(post_ids)):
    if post_contents[i] != '[removed]' and post_contents[i] != '[deleted]':
        post_ids_contents.append([post_ids[i], post_contents[i]])
    if post_titles[i] != '[deleted by user]':
        post_ids_titles.append([post_ids[i], post_titles[i]])

In [21]:
comment_ids = list(result_df['comment_id'])
comment_bodies = list(result_df['comment_body'])

parent_ids_comment_ids_bodies = []

for i in range(len(comment_ids)):
    if comment_bodies[i] != '[removed]' and comment_bodies[i] != '[deleted]':
        parent_ids_comment_ids_bodies.append([post_ids[i], comment_ids[i], comment_bodies[i]])

# Counting sentences

In [12]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/mykim/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [30]:
post_contents_len_list = []
comment_bodies_len_list = []


for p_pair, c_pair in zip(post_ids_contents, parent_ids_comment_ids_bodies):
    post_contents_len_list.append(len(sent_tokenize(p_pair[1])))
    comment_bodies_len_list.append(len(sent_tokenize(c_pair[2])))
    # print(len(sent_tokenize(pair[1])))
    
print(post_contents_len_list)
print(comment_bodies_len_list)


[15, 11, 6, 20, 9, 8, 10, 13, 7, 7, 2, 12, 9, 2, 19, 4, 4, 5, 13, 7, 8, 6, 7, 7, 7, 7, 6, 9, 11, 7, 5, 3, 3, 3, 5, 2, 8, 7, 12, 3, 1, 7, 4, 4, 9, 4, 10, 3, 1, 8, 4, 6, 3, 11, 6, 5, 12, 2, 4, 3, 10, 8, 10, 1, 13, 4, 8, 8, 2, 22, 10, 7, 6, 12, 3, 6, 15, 11, 9, 10, 1, 3, 4, 4, 7, 7, 7, 9, 6, 4, 9, 8, 5, 3, 4, 3, 4, 3, 8, 12, 11, 4, 10, 15, 10, 10, 11, 16, 5, 7, 3, 11, 5, 11, 10, 2, 1, 17, 9, 3, 3, 7, 4, 10, 8, 11, 6, 14, 10, 11, 12, 5, 2, 17, 5, 8, 7, 8, 3, 7, 6, 2, 6, 5, 10, 3, 7, 12, 3, 4, 4, 2, 10, 4, 16, 11, 11, 5, 4, 19, 15, 12, 17, 2, 8, 6, 18, 9, 10, 3, 12, 5, 4, 12, 3, 2, 6, 1, 14, 3, 1, 9, 5, 2, 16, 9, 15, 10, 16, 23, 27, 11, 9, 5, 8, 10, 22, 9, 4, 7, 14, 5, 13, 2, 10, 8, 7, 13, 12, 5, 3, 5, 7, 16, 3, 6, 4, 4, 7, 2, 23, 17, 16, 3, 9, 9, 9, 8, 6, 8, 8, 2, 8, 4, 7, 13, 6, 5, 7, 5, 10, 4, 4, 15, 3, 16, 20, 5, 12, 12, 10, 12, 4, 4, 4, 6, 6, 11, 5, 19, 6, 1, 17, 3, 5, 5, 12, 14, 4, 11, 9, 10, 8, 3, 8, 3, 8, 7, 7, 6, 3, 3, 3, 11, 6, 11, 12, 3, 2, 2, 2, 10, 3, 5, 11, 5, 13, 5, 4, 4, 9, 

In [32]:
print(f'Average sentence count of post contents: {sum(post_contents_len_list) / len(post_contents_len_list)}')
print(f'Max count of post contents: {max(post_contents_len_list)}')
print(f'average sentence count of comment bodies: {sum(comment_bodies_len_list) / len(comment_bodies_len_list)}')
print(f'Max count of comment bodies: {max(comment_bodies_len_list)}')

Average sentence count of post contents: 7.621794871794871
Max count of post contents: 27
average sentence count of comment bodies: 3.3141025641025643
Max count of comment bodies: 10


In [19]:
new_post_ids_contents = []

for l in post_ids_contents:
    if l not in new_post_ids_contents:
        new_post_ids_contents.append(l)

In [18]:
len(new_post_ids_contents)

621

In [20]:
def tf_idf_similarity_from_list(document):
    document = check_body(document)

    # comment/body list가 빈 list인지를 체크해서 빈 list이면 -1 return.
    if not document:
        return -1
    
    # check_body() 함수를 거친 후, 이모지로만 이루어진 comment/post title이 지워져서
    # 하나의 text만 남았을 경우. 구분을 위해 -2 return.
    if len(document) == 1:
        return -2

    tfidf_vectorizer = TfidfVectorizer(min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform(document)

    document_distances = (tfidf_matrix * tfidf_matrix.T)
    
    result_array = document_distances.toarray()
    
    similarity = 0
    count = 0
    
    for i in range(len(result_array)):
        for j in range(len(result_array[i])):
            if i < j:
                similarity += result_array[i][j]
                count += 1
    
    return (similarity / count)

In [78]:
for i in range(5):
    for j in range(i, 5):
        if i != j:
            print(i, j)

0 1
0 2
0 3
0 4
1 2
1 3
1 4
2 3
2 4
3 4


In [21]:
post_content_result_list = []

In [22]:
for i in range(len(new_post_ids_contents)):
    print(i, end=' ')
    for j in range(i, len(new_post_ids_contents)):
        if i != j:
            post_content_result_list.append([new_post_ids_contents[i][0], new_post_ids_contents[j][0], tf_idf_similarity_from_list([new_post_ids_contents[i][1], new_post_ids_contents[j][1]])])

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [83]:
titles_result_list = []

In [84]:
for i in range(len(post_ids_titles)):
    print(i, end=' ')
    for j in range(i, len(post_ids_titles)):
        if i != j:
            titles_result_list.append([post_ids_titles[i][0], post_ids_titles[j][0], tf_idf_similarity_from_list([post_ids_titles[i][1], post_ids_titles[j][1]])])

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [34]:
print(post_ids_contents[0][1])
print(post_ids_contents[1][1])

Third year college student. My depression has always stemmed largely from social anxiety, wishing I had friends but not knowing how to make them. I spent two years in dorms and never spoke to any of my neighbors and didn't really meet anybody.

Last night, for the first time in my life, I was invited to a party (not personally mind you; I was a part of a group that got collectively invited). I drank a little bit, but got tipsy quickly, and not knowing my limits I stopped. I was completely sober within the first hour. 

Besides introductions and an occasional greeting, the only people I actually spoke to were the people I already knew. I then spent a few hours sitting in a corner and observing drunk antics.

It was pretty weird, observing two people go from complete strangers, to being in one another's arms at the end of the night. Looks so easy.

Is this the shit that I've been "missing" for all of my life? In which case, wow. Good riddance, I'll take my depression over that any day.



In [55]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   post_index             1000 non-null   int64  
 1   author_id              1000 non-null   object 
 2   post_id                1000 non-null   object 
 3   post_title             1000 non-null   object 
 4   post_utc               1000 non-null   int64  
 5   post_num_comments      1000 non-null   int64  
 6   post_score             1000 non-null   int64  
 7   post_upvote_ratio      1000 non-null   float64
 8   post_content           1000 non-null   object 
 9   post_tenure            1000 non-null   int64  
 10  familiarity            1000 non-null   int64  
 11  post_IS                1000 non-null   int64  
 12  post_ES                1000 non-null   int64  
 13  comment_id             1000 non-null   object 
 14  comment_body           1000 non-null   object 
 15  comme

In [28]:
print(len(parent_ids_comment_ids_bodies))

882


In [30]:
new_parent_ids_comment_ids_bodies = []

for l in parent_ids_comment_ids_bodies:
    if l not in new_parent_ids_comment_ids_bodies:
        new_parent_ids_comment_ids_bodies.append(l)

In [31]:
print(len(new_parent_ids_comment_ids_bodies))

882


In [85]:
comments_result_list = []

In [86]:
for i in range(len(parent_ids_comment_ids_bodies)):
    print(i, end=' ')
    for j in range(i, len(parent_ids_comment_ids_bodies)):
        if i != j:
            comments_result_list.append([parent_ids_comment_ids_bodies[i][0], parent_ids_comment_ids_bodies[i][1], 
                                       parent_ids_comment_ids_bodies[j][0], parent_ids_comment_ids_bodies[j][1], 
                                       tf_idf_similarity_from_list([parent_ids_comment_ids_bodies[i][2], parent_ids_comment_ids_bodies[j][2]])])

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [26]:
# make csv files here
# post_content_result_list, titles_result_list, comments_result_list
header = ['post_id1', 'post_id2', 'tf-idf_similarity']
post_content_df = pd.DataFrame(post_content_result_list, columns = header)
post_content_df.to_csv(f"../predicting-satisfaction-using-graphs/csv/tf_idf_similarity_post_contents_pair.csv", header=header, index=None)

In [23]:
post_titles_df = pd.DataFrame(titles_result_list, columns = header)
post_titles_df.to_csv(f"../csv/tf_idf_similarity_post_titles_pair.csv", header=header, index=None)

NameError: name 'titles_result_list' is not defined

In [91]:
header = ['parent_id1', 'comment_id1', 'parent_id2', 'comment_id2', 'tf-idf_similarity']
comments_df = pd.DataFrame(comments_result_list, columns = header)
comments_df.to_csv(f"../temp/csv/tf_idf_similarity_comments_pair.csv", header=header, index=None)

# BERT-NER

In [94]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [110]:
example = post_contents[5]
print(example)


ner_results = nlp(example)
print(ner_results)

Hello. I have had insecurities, anxiety, depression and trust issues all my life. Between September and December I started smoking pot very often. I stopped for awhile and  started again in January. I had a problem smoking everyday to the point where I felt dead inside, emotionless. I felt like nothing was real. I haven't smoked since February, but I have had some personal issues and I'm trying to develop into a better person. Sometimes I get so depressed that I have suicidal thoughts and don't eat. I still feel like nothing is real at times. I feel disconnected from everyone and everything, even myself. I started going to therapy but I need more help. I am just so confused about everything to the point where it scares me. I don't know how to explain but it's like my subconscious mind is in control. It's like I'm in the "sunken place" (Get Out reference). I feel like I don't understand myself or anything for that matter. Life is so confusing to me I just feel so alone. Can anyone expla

In [114]:
print(ner_results[0]['word'])

Out


In [107]:
for content in post_ids_contents:
    # print(content[1])
    ner_results = nlp(content[1])
    print(ner_results)
    print('///')

Third year college student. My depression has always stemmed largely from social anxiety, wishing I had friends but not knowing how to make them. I spent two years in dorms and never spoke to any of my neighbors and didn't really meet anybody.

Last night, for the first time in my life, I was invited to a party (not personally mind you; I was a part of a group that got collectively invited). I drank a little bit, but got tipsy quickly, and not knowing my limits I stopped. I was completely sober within the first hour. 

Besides introductions and an occasional greeting, the only people I actually spoke to were the people I already knew. I then spent a few hours sitting in a corner and observing drunk antics.

It was pretty weird, observing two people go from complete strangers, to being in one another's arms at the end of the night. Looks so easy.

Is this the shit that I've been "missing" for all of my life? In which case, wow. Good riddance, I'll take my depression over that any day.



[]
///
I have finals next week and I simply can't even study for them or wake up and read a book. I have been trying for days and days. I might be able to go to the library one day, but next day I won't even move from bed, and the cycle keeps happening. 

I have so much riding on this next week, it is so important for me to do well yet I just don't do it, I just browse Reddit and imgur. It's not even too difficult, I can take it in small chunks if I could manage it. 

I'm on meds so at least I'm able to shower and eat now (unlike a couple months ago) but still this progress is way too slow. 

I really don't know what's wrong with me, it's like seeing a train come towards you and not moving out the way when you know you could so easily move. Every day that train comes closer. Everyday I don't move. 
[{'entity': 'B-MISC', 'score': 0.5585535, 'index': 95, 'word': 'Red', 'start': 365, 'end': 368}, {'entity': 'I-MISC', 'score': 0.67244595, 'index': 96, 'word': '##dit', 'start': 368, 'end': 

[]
///
My name is Rémy. I took part in the war in Mali and was in Paris, my hometown, after the November attacks. The shock of that event really messed me up in many ways, and I also have PTSD from war experiences. 

Luckily I mostly recovered and I'm able to live my life, but every day I'm depressed. Even though I'm surrounded by people I feel alienated and lost. I don't know if there's anyone here to talk to, I hope so. 

Kindest regards,
Rémy
[{'entity': 'B-PER', 'score': 0.99747664, 'index': 4, 'word': 'R', 'start': 11, 'end': 12}, {'entity': 'B-PER', 'score': 0.8832674, 'index': 5, 'word': '##é', 'start': 12, 'end': 13}, {'entity': 'B-PER', 'score': 0.94591564, 'index': 6, 'word': '##my', 'start': 13, 'end': 15}, {'entity': 'B-LOC', 'score': 0.9998104, 'index': 15, 'word': 'Mali', 'start': 43, 'end': 47}, {'entity': 'B-LOC', 'score': 0.99973565, 'index': 19, 'word': 'Paris', 'start': 59, 'end': 64}, {'entity': 'B-PER', 'score': 0.99722075, 'index': 112, 'word': 'R', 'start': 438, 

[]
///
I got into a pretty bad car accident today and my mom picked me up and helped me with talking to the cops and everything. On the drive home I couldn't stop crying even though no one was hurt and mostly everything was taken care of. 

My mom kept asking if there was more going on and after keeping it to myself for over 4 years I finally told her I'm depressed.  

It was so heartbreaking seeing how sad she was in her eyes. I know it's not my fault for having depression but I feel awful hurting her like that. I know she blames her self and I tried to make it very clear that that was not the case. 

I promised her I'd start going to therapy and I plan to follow through on that. I just hope I can get better for her and me and that our relationship will be good. 

She tried to talk to me about what may be causing it while we were in the car and I don't know exactly so I just kinda listed some things I  struggle with because of it like social anxiety, loneliness, and horrible sleeping 

[{'entity': 'B-LOC', 'score': 0.9997937, 'index': 80, 'word': 'Georgia', 'start': 334, 'end': 341}, {'entity': 'B-ORG', 'score': 0.9919594, 'index': 117, 'word': 'SF', 'start': 498, 'end': 500}]
///
I feel like I'm stuck in a continuous loop I'm sick of my job I hate my fake friends I hate just about everything, If I had a gun I would put a bullet through my skull and end this cold, empty meaningless existence. 
[]
///
I am separated (which devastates me). I have my son this weekend. I'm stuck in bed. I *love* him and I'm so happy when he's here. But it's still hard to get up. 

I do better at night. He notices that. He also knows of my depression. I can't hide it and its nothing to feel ashamed of. I don't tell him I'm suicidal of course. I don't tell him I want to die, I just say "depression sucks".

Edit: Big thank you to everyone. Thank God there is somewhere I can talk when I feel like I'm going to explode or break down.
[]
///
I was a 21 year old student studying physics at the u

...

[]
///
My job is a dreaded, miserable, soul crushing, boring, never ending, depression causing, destroyer of my life. 

I can't quit because money. And it's by far easier in my field to get a job while still employed than to explain a gap or why I left. I don't know how long it will take to actually find one so there is no guarantee how short the gap would be if I quit. If I quit, I lose almost all negotiation power leveraging my current job. 

On the other hand, this job is ruining my life and it scares me to think about the person I will become if I am trapped here another 6 months, year, 2 years. I just can't. 

But the job search is excruciatingly slow. Impossible. Demoralizing. Unfair. I hate it almost as much. 

So here I am trapped and unable to get out and try to better myself and my mental health. 
[]
///
I want you guys to know that I made myself breakfast...or dinner, or whatever. The point is I got up and made myself food. Two eggs, two pieces of sausage, and a piece of bre

[]
///
I know that seems like a stupid question, but I feel like I'm really not worth it. I don't even know if I have depression, but I do feel empty most of the time. I also know that I'm a bad human being, and that isn't depression talking, that's just common sense. I know it seems like this is a scenario where my lack of insight is clouding my judgement of myself, but my insight is crystal clear.

On top of this I believe I'm losing my mind. Somebody will tell me something and then I'll forget it a few seconds later, or I won't be able to connect to things. Kinda like having a conversation like:

**Friend:** We had pizza for dinner last night.

**Me:** Sounds good, what'd you have?

I think I might be retarded or something. No matter how hard I try I can't stop being creepy. 

And that example of a conversation, while accurate to my horrible social skills, may make someone think I actually have friends, which I don't. Which is relevant because I'm not sure if I'm stupid or just have

[]
///
Everyday I try so hard to get out of bed and smile.... It's never enough.

Everyday I try not to get overwhelmed and look professional at work and respect my work polices... I still can't do it right and end up crying. 

Everyday I try to be productive and prudent... and I fail. 

My psychiatrist says my expectations are so high I can't please them. But I just feel like I'm doing nothing right and I'm going to lose my fulltime job and friends. 

How are you suppose to keep trying when every possible little mistake  leads to my future going up in smoke? 

I am supposed to live in the present so I don't get overwhelmed by these thing and trust me I try but I can't help but fear that it's going to turn against me like everything else has.

Sorry for venting. Just don't know what else to do. 


[]
///
I have a really clingy personality, so the few girlfriends I've had I ended up pushing away.  I have a problem where I center my life too much around them.  I'm aware and have been try

[]
///
I've been dating a really, really nice guy for a few weeks now. We met at a friend's party, and we just hit it off. We're just getting to know each other. He's extremely intelligent, super-cute, very driven and ambitious, and he gives a shit about the world at large. He laughs easily but can be a little awkward :) (gosh, am I smitten or what) He's a bit of a reserved person - I'm putting that down to him only just learning to opening up to me - while I'm an outgoing, bare-all sort of person. He's been thoughtful and patient so far, and it's really been sweet.

Ooh, and, we're doing long distance. So you can see, we'd both like for this to work.

At what point is it ok to let him know that I'm a manic depressive, prone to bouts of intense anxiety?

It's not that I'm afraid he'll judge me (that would totally make him dumpster material) - but when you reveal something like this to a person you're dating, it's kinda stating an expectation. It's a deeply personal struggle, and it mea

[{'entity': 'B-MISC', 'score': 0.53815883, 'index': 4, 'word': 'T', 'start': 20, 'end': 21}, {'entity': 'I-MISC', 'score': 0.9748514, 'index': 5, 'word': 'Shi', 'start': 22, 'end': 25}, {'entity': 'I-MISC', 'score': 0.60600346, 'index': 6, 'word': '##rts', 'start': 25, 'end': 28}]
///
Shouldn't  my supposed worth and value be reflected through my experiences in life and how others perceive me? Can I really blame my sadness on bad luck? I think not. 
[]
///


In [108]:
print(ner_results)

[]


In [126]:
print(len(post_ids_contents))
print(len(post_ids_titles))
print(len(parent_ids_comment_ids_bodies))

624
999
882


In [127]:
post_contents_ner = []
post_titles_ner = []
comment_bodies_ner = []

for i in range(len(post_ids_contents)):
    # print(content[1])
    ner_results = nlp(post_ids_contents[i][1])
    s = set()
    # print(ner_results)
    for entity in ner_results:
        s.add(entity['word'])
    if s:
        print(s)
    
    post_contents_ner.append([post_ids_contents[i][0], s])

print(len(post_contents_ner))
    
    
for i in range(len(post_ids_titles)):
    # print(content[1])
    ner_results = nlp(post_ids_titles[i][1])
    s = set()
    # print(ner_results)
    for entity in ner_results:
        s.add(entity['word'])
    if s:
        print(s)
    
    post_titles_ner.append([post_ids_titles[i][0], s])
print(len(post_titles_ner))
    
    
for i in range(len(parent_ids_comment_ids_bodies)):
    # print(content[1])
    ner_results = nlp(parent_ids_comment_ids_bodies[i][2])
    s = set()
    # print(ner_results)
    for entity in ner_results:
        s.add(entity['word'])
    if s:
        print(s)
    
    comment_bodies_ner.append([parent_ids_comment_ids_bodies[i][0], parent_ids_comment_ids_bodies[i][1], s])
print(len(comment_bodies_ner))

{'Out'}
{'Red', '##dit'}
{'Catholic'}
{'R', '##é', '##my', 'Mali', 'Paris'}
{'##IB', 'S'}
{'IS', '##IS'}
{'Red', '##dit'}
{'Georgia', 'SF'}
{'Ryan'}
{'Ty'}
{'LA'}
{'England'}
{'Red', 'N', '##W', '##dit', '##SF'}
{'Ph', '##ener'}
{'Shane', '##ell', 'Hard', 'Better', 'Ash', 'Dawson', 'Bobby'}
{'##i', 'Un'}
{'American', 'Ethiopian'}
{'Non'}
{'Doc'}
{'God', 'G'}
{'C', '##op', '##ram', '##ital'}
{'the', 'Keep', 'Score', '##SD', '##s', 'The', 'D', 'Body', 'PT'}
{'Red', '##dit'}
{'Everest'}
{'McDonald', "'", 's'}
{'##bu', 'Star', '##cks', 'Vegas'}
{'s', 'Kyle', 'World', 'Greatest', 'Dad', "'"}
{'Thomas'}
{'I'}
{'##olo', 'Z'}
{'O', '##TC', 'Amtrak'}
{'Western'}
{'P', 'G'}
{'Netflix'}
{'YouTube'}
{'##cha', '##nap', 'S', '##t'}
{'Depression'}
{'History', 'European', 'AP'}
{'##B'}
{'America'}
{'Baker', 'Act'}
{'A', '##Z'}
{'Cam', 'Co', 'and', '##d', '##bria', '##hee'}
{'God'}
{'A', 'P', 'S'}
{'Python'}
{'Red', '##dit'}
{'God'}
{'SS', '##dic', 'Me', '##are', 'Di', 'Iraq', '&', '##sa', '##bility'}


In [120]:
print(parent_ids_comment_ids_bodies[0])

['112354', 'c6irl1u', 'One thing I am going to commend you on not excessively drinking and showing restraint. Drinking is usually associated with social activities but the tradeoff of being a jackass is not worth it.']


In [160]:
ner_titles_result = []
ner_contents_result = []
ner_comments_result = []

In [161]:
for i in range(len(post_titles_ner)):
    print(i, end=' ')
    for j in range(i, len(post_titles_ner)):
        if i != j:
            jaccard = 0.0
            if len(post_titles_ner[i][1] | post_titles_ner[j][1]) != 0:
                jaccard = len(post_titles_ner[i][1] & post_titles_ner[j][1]) / len(post_titles_ner[i][1] | post_titles_ner[j][1])
            ner_titles_result.append([post_titles_ner[i][0], post_titles_ner[j][0],
                                     len(post_titles_ner[i][1] & post_titles_ner[j][1]),
                                     jaccard])
        # print(jaccard)
        # print(ner_titles_result)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [162]:
for i in range(len(post_contents_ner)):
    print(i, end=' ')
    for j in range(i, len(post_contents_ner)):
        if i != j:
            jaccard = 0
            if len(post_contents_ner[i][1] | post_contents_ner[j][1]) != 0:
                jaccard = len(post_contents_ner[i][1] & post_contents_ner[j][1]) / len(post_contents_ner[i][1] | post_contents_ner[j][1])
            ner_contents_result.append([post_contents_ner[i][0], post_contents_ner[j][0],
                                     len(post_contents_ner[i][1] & post_contents_ner[j][1]),
                                     jaccard])
        # print(jaccard)
        # print(ner_titles_result)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [163]:
for i in range(len(comment_bodies_ner)):
    print(i, end=' ')
    for j in range(i, len(comment_bodies_ner)):
        if i != j:
            jaccard = 0.0
            if len(comment_bodies_ner[i][2] | comment_bodies_ner[j][2]) != 0:
                jaccard = len(comment_bodies_ner[i][2] & comment_bodies_ner[j][2]) / len(comment_bodies_ner[i][2] | comment_bodies_ner[j][2])
            ner_comments_result.append([comment_bodies_ner[i][0], comment_bodies_ner[i][1],
                                        comment_bodies_ner[j][0], comment_bodies_ner[j][1],
                                     len(comment_bodies_ner[i][2] & comment_bodies_ner[j][2]),
                                     jaccard])
        # print(jaccard)
        # print(ner_titles_result)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [145]:
comment_bodies_ner[12][2]

set()

In [146]:
for n in comment_bodies_ner:
    if n[2]:
        print(n[2])

{'Political', 'Science'}
{'D'}
{'E', 'P', '##rist', '##iq'}
{'Ser', 'Z', '##tral'}
{'S'}
{'O'}
{'State', 'the', 'Union', 'of'}
{'E'}
{'##ony', '##mous', 'An'}
{'Stevens'}
{'Je'}
{'Netflix'}
{'NHS', '111'}
{'P', '##HP', '##OP', 'I'}
{'Thor'}
{'i'}
{'EU', 'T', '##RO'}
{'U'}
{'E'}
{'Mind'}
{'Ty'}
{'Rivers', 'Plains', 'Rockies', 'Rocky', 'Seattle'}
{'##ort', 'St', 'Johns', 'W'}
{'World', 'Jimmy', 'Eat'}
{'O'}
{'T', 'Catholicism'}
{'Illinois'}
{'City', 'New', 'York'}
{'AG', '##lad', 'Mu', 'Steam', '##s', '##C'}
{'Harvard', 'Li', 'AP', '##t'}
{'O'}
{'##olo', 'Ce', 'Z', '##lex', '##a'}
{'Jim', '##ck', 'Hu'}
{'O'}
{'New', 'York'}
{'Christianity'}
{'Health', 'Florida', 'Department'}
{'Python'}
{'B'}
{'Korean'}
{'##zac', 'Pro'}
{'N'}
{'SSR', '##I'}
{'Satan'}
{'##W', 'Matrix', 'BT'}
{'Indian', 'Indies', 'Australian', 'West', 'British'}
{'Depression'}
{'##z', 'Re', 'Trent', '##nor'}
{'Po', '##tten'}
{'Bad', 'Po', 'Simon', 'Day', 'G', 'Daniel', 'Water', 'Bridge', 'over', '##ter', '##ar', 'troubled'

In [151]:
for n in ner_comments_result:
    if n[5]:
        print(n)

['32ovjj', 'cqdd9ed', '7l6dgb', 'drk23o4', 1, 1.0]
['a9ssn4', 'ecma6g3', '8gmwdq', 'dyd31sz', 1, 0.25]
['a9ssn4', 'ecma6g3', 'a0h8c0', 'eahpsg3', 1, 0.14285714285714285]
['a9ssn4', 'ecma6g3', 'a46ol8', 'ebby72h', 1, 0.25]
['a9ssn4', 'ecma6g3', 'bug2gr', 'epbmmih', 1, 0.2]
['a9ssn4', 'ecma6g3', '4afk2c', 'd100vy3', 1, 0.045454545454545456]
['b7994c', 'ejqb4x8', 'b34ka7', 'eixb98r', 1, 0.14285714285714285]
['b7994c', 'ejqb4x8', '4risp9', 'd51nj1n', 1, 0.2]
['hc3gl', 'c1u9e5p', '14m6n4', 'c7edqb3', 1, 0.25]
['hc3gl', 'c1u9e5p', 'n8z58', 'c378if9', 1, 1.0]
['hc3gl', 'c1u9e5p', 'bug2gr', 'epbmmih', 1, 0.5]
['hc3gl', 'c1u9e5p', '4afk2c', 'd100vy3', 1, 0.05263157894736842]
['4qvtbc', 'd4wdudf', '2xx8g5', 'cp4dfjx', 1, 1.0]
['4qvtbc', 'd4wdudf', '7tyrk6', 'dtg9xld', 1, 1.0]
['4qvtbc', 'd4wdudf', '5nyv1d', 'dcfobfs', 1, 1.0]
['2t3bc2', 'cnvmrw8', '193e2d', 'c8kj6ip', 1, 0.07692307692307693]
['2t3bc2', 'cnvmrw8', '6y7i0t', 'dml93ne', 1, 0.14285714285714285]
['2t3bc2', 'cnvmrw8', '1hntve', 'caw3p

In [164]:
# make csv files here
# ner_titles_result, ner_contents_result, ner_comments_result
header = ['post_id1', 'post_id2', 'number_of_overlapped_words', 'jaccard_coefficient']
ner_titles_df = pd.DataFrame(ner_titles_result, columns = header)
ner_titles_df.to_csv(f"../temp/csv/ner_jaccard_post_titles_pair.csv", header=header, index=None)

ner_contents_df = pd.DataFrame(ner_contents_result, columns = header)
ner_contents_df.to_csv(f"../temp/csv/ner_jaccard_post_contents_pair.csv", header=header, index=None)

In [166]:
header = ['parent_id1', 'comment_id1', 'parent_id2', 'comment_id2', 'number_of_overlapped_words', 'jaccard_coefficient']
ner_comments_df = pd.DataFrame(ner_comments_result, columns = header)
ner_comments_df.to_csv(f"../temp/csv/ner_jaccard_comments_pair.csv", header=header, index=None)